In [1]:
main_amenity = "golf_course"
dist_in_meters=100

In [2]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [3]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=places/type=place/"
)

In [4]:
df.schema

SedonaSchema with 14 fields:
  id: utf8<Utf8View>
  geometry: geometry<WkbView(ogc:crs84)>
  bbox: struct<Struct(xmin Float32, xmax Float32, ymin Float32, ymax Float32)>
  version: int32<Int32>
  sources: list<List(Field { name: "element", data_type: Struct([Field { name: "property", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "dataset", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "record_id", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "update_time", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "confidence", data_type: Float64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "between", data_type: List(Field { name: "element", data_type: Float64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id

In [5]:
df.to_view("places", True)

In [6]:
sd.sql(f"""
SELECT 
    geometry as geom,
    addresses[1]['locality'] AS locality
FROM places
WHERE 
    categories.primary = '{main_amenity}'
    AND addresses[1]['country'] = 'US'
""").to_view("main", True)

In [7]:
sd.sql(f"""
SELECT 
    geometry as geom,
    categories.primary AS cat
FROM places
WHERE 
    array_has(['day_spa', 'boxing_gym', 'bubble_tea'], categories.primary)
    AND addresses[1]['country'] = 'US'
""").to_view("amenities", True)

In [8]:
sd.sql(f"""
SELECT 
    COUNT(a.*) AS proximity_count, locality, array_distinct(array_agg(cat)) 
FROM main g 
JOIN amenities a ON ST_Distance(g.geom, a.geom) <= {dist_in_meters}
group by locality 
order by 1 DESC 
""").show(10)

KeyboardInterrupt: 